** what I want to do : ** In the previous notebook, I built prototype of Tfidf summaries simply getting the sentence that contained the phrase with highest tf-idf for the first time. The result was not very good! Some changes I have in the pipeline are: 
    - REMEMBER TO CHANGE \N IN THE CLEANSE PROCESS
    - Include more whitepapers to make the idf more tenable : DONE
    - GET WEIGHTED TF-IDF OF WHOLE SENTENCE AS DESCRIBED IN http://research.nii.ac.jp/ntcir/workshop/OnlineProceedings3/NTCIR3-TSC-SekiY.pdf. USE https://medium.com/@acrosson/summarize-documents-using-tf-idf-bdee8f60b71

## Admin and Imports

In [1]:
# the usual suspects...
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import itertools
import re

# this is for my bitcoin prices
import quandl

# visualisation libraries: I'll go ahead and use plotly (offline) for graphing because its d3 
# and so I can track prices over time
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as sct
import seaborn as sns
sns.set()
sns.set(style="ticks")
sns.set_color_codes("muted")

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
pd.set_option('display.float_format', lambda x:'%.10f' %x)
import glob

# we'll need this for plotting in nice colours. Trust me on this.
COLOR_PALETTE = [    
               "#348ABD",
               "#A60628",
               "#7A68A6",
               "#467821",
               "#CF4457",
               "#188487",
               "#E24A33"
              ]

## Read Data

In [39]:
df_white = pd.read_json('tldr_whitepaper_raw.json', orient='index')
df_white

,body,summary
agrello,1 Introduction\n The traditional understand...,This whitepaper addresses existing problems wi...
blackmoon,VISION\n Blackmoon Crypto is part of the Blac...,Blackmoon Crypto is part of the Blackmoon Fina...
coindash,Introduction\n One of most potentially transf...,As the blockchain industry expands and the num...
delphi,Introduction\n Prediction Markets\n ...,\n Delphi represents the next evolutionar...
ethereum,,
filecoin,1 Introduction Filecoin is a protocol ...,The internet is in the middle of a revolution:...
gnosis,1.1 Problem Overview Generally speak...,Prediction markets are poised to become one of...
iota,,
neblio,Blockchain\n What is the Neblio Blockchain?\n...,Blockchain’s unique distributed ledger technol...
numerai,,


In [41]:
df_white = df_white.applymap(lambda x: np.nan if x=='' else x)